In [2]:
import pickle
import pandas as pd

In [21]:
eval_traffic = "/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/evaluation_datasets/V2_eval_trafficDataset_2024-3-14_1645hours_1296000steps.csv"
eval_vehicle = "/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/evaluation_datasets/V2_eval_vehicleDataset_2024-3-14_1645hours_1296000steps.csv"

NUM_OF_JUNCTIONS = 8
ROLLING_WINDOW_LENGTH = 300
TRAVEL_TIME_WINDOW = 300
LENGTH_OF_SIM = 1296000
generate_incidents_csv = True

In [5]:
print(f"Number of junctions considered for dataset prep: {NUM_OF_JUNCTIONS}")
print(f"Rolling window length considered for dataset prep: {ROLLING_WINDOW_LENGTH}")
print(f"Travel time window length considered for dataset prep: {TRAVEL_TIME_WINDOW}")

Number of junctions considered for dataset prep: 8
Rolling window length considered for dataset prep: 300
Travel time window length considered for dataset prep: 300


In [7]:
dtype = {
    'step' : 'int64',
    'time_of_day' : 'int64',
    'identified_edge' : 'object',
    'junction_mean_speed' : 'float64',
    'traffic_count' : 'float64',
    'traffic_occupancy' : 'float64',
    'vehicles_per_lane_1' : 'int64',
    'vehicles_per_lane_0' : 'int64',
    'lane_mean_speed_0' : 'float64',
    'lane_mean_speed_1' : 'float64',
    'incident_edge': 'object',
    'incident_start_time': 'float64',
    'incident_type': 'object',
    'accident_label': 'bool',
    'accident_id': 'object',
    'accident_duration': 'float64',
    'incident_lane': 'object'
}
df_traffic = pd.read_csv(eval_traffic,dtype=dtype)

In [8]:
df_traffic.head()

,step,time_of_day,identified_edge,junction_mean_speed,traffic_count,traffic_occupancy,vehicles_per_lane_1,vehicles_per_lane_0,lane_mean_speed_0,lane_mean_speed_1,incident_edge,incident_start_time,incident_type,accident_label,accident_id,accident_duration,incident_lane,junction
0,0,0,533573776#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
1,0,0,436794680#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
2,0,0,256917837#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
3,0,0,436791498#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
4,0,0,436794701#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1


In [9]:
INCOMING_EDGES = ['436794670','436791113#0','436794680#0','436940278','-643913497','436943745#0','351673438','-613687451#1','436943750#0','436943762#0','-436942356#1','436942362#0','436789580#1','436942357','436794669','1051038541#0','-436794679#3','533371302#0','436942374','436790491','-436942381#3']

OUTGOING_EDGES = ['533573776#0','5607328#0','519448767','436940270','-436943745#2','436943742','436943774','436943743#0','-436943762#2','436942356#0','-436942362#3','436789564#0','531969915#0','436794679#0','-1088637809#1','436942385#0','-436794676#1','436790495','-1033824750','30031286#0','436942381#0']


In [10]:
sensors = INCOMING_EDGES+OUTGOING_EDGES
print(len(sensors))

42


In [11]:
sensors_more = {    1:["533573776#0","436794680#0","256917837#0","436791498#0","436794701#0","-436794701#3","5607328#0","436791113#0"],
    2:["436794679#0","-436794679#3","436942387#0","436942371#0","-1088637809#1","436794670","436942385#0","1051038541#0"],
    3:["436942349#0","-436942349#3","-403117287#3","403117287#0","-436794676#1","436794669","531969915#0","436942357"],
    5:["30031286#0","436790491","436942381#0","-436942381#3","436790495","533371302#0","-1033824750","436942374"],
    6:["422264711#0","422264712#0","-436942362#3","436942362#0","436789564#0","436789580#1","436942356#0","-436942356#1"],
    7:["436940270","-643913497","519448767","436940278","436940284","-597312446","422267251","422267252"],
    8:["436943742","351673438","436943774","-613687451#1","-436943745#2","436943745#0","-436943776#4","436943776#0"],
    9:["406379332","436943746#0","-436943762#2","436943762#0","436943743#0","436943750#0","436943757#0","-436943757#1"]
}

excess_sensors = []
for key in sensors_more.keys():
    
    excess_sensors = excess_sensors + sensors_more[key]

In [14]:
set_diff = set(excess_sensors).difference(set(sensors))
print(set_diff)
print(len(set_diff))

{'436942371#0', '436942349#0', '-436943757#1', '436940284', '422267251', '-436794701#3', '-403117287#3', '-597312446', '436794701#0', '256917837#0', '-436942349#3', '436943757#0', '403117287#0', '422264711#0', '422267252', '406379332', '436942387#0', '436791498#0', '436943746#0', '422264712#0', '436943776#0', '-436943776#4'}
22


In [15]:
excess_list = list(set_diff)
print(excess_list)

['436942371#0', '436942349#0', '-436943757#1', '436940284', '422267251', '-436794701#3', '-403117287#3', '-597312446', '436794701#0', '256917837#0', '-436942349#3', '436943757#0', '403117287#0', '422264711#0', '422267252', '406379332', '436942387#0', '436791498#0', '436943746#0', '422264712#0', '436943776#0', '-436943776#4']


In [17]:
df_traffic_reduced = df_traffic[~df_traffic['identified_edge'].isin(excess_list)]

In [18]:
df_traffic_reduced.head()

,step,time_of_day,identified_edge,junction_mean_speed,traffic_count,traffic_occupancy,vehicles_per_lane_1,vehicles_per_lane_0,lane_mean_speed_0,lane_mean_speed_1,incident_edge,incident_start_time,incident_type,accident_label,accident_id,accident_duration,incident_lane,junction
0,0,0,533573776#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
1,0,0,436794680#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
6,0,0,5607328#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
7,0,0,436791113#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,1
8,0,0,436794679#0,0.0,0.0,0.0,0,0,0.0,0.0,NaN,0.0,NaN,False,NaN,0.0,-1,2


In [19]:
df_traffic.shape

(82944000, 18)

In [20]:
df_traffic_reduced.shape

(54432000, 18)

In [22]:
if(generate_incidents_csv):
    print("Generating incidents csv")
    df_incidents_info = df_traffic_reduced[df_traffic_reduced["accident_label"]][["incident_edge", "incident_type", 'incident_start_time',"accident_duration", "incident_lane"]]

    df_incidents_info2 = df_incidents_info.groupby("incident_start_time").first().reset_index()

    print("Getting head of incidents dataset")

    print(df_incidents_info2.head())


    df_incidents_info2.to_csv("incidents_data.csv")

    print(f"Number of incidents in this simulation {len(df_incidents_info2)}")
    print(f"Number of incidents in this simulation with less severe incidents {len(df_incidents_info2[df_incidents_info2['incident_type']=='stalled_vehicle'])}")
    print(f"Number of incidents in this simulation with more severe incidents {len(df_incidents_info2[df_incidents_info2['incident_type']=='multi_vehicle_collision'])}")

    print(f"Input traffic raw data shape is {df_traffic.shape}")

Generating incidents csv
Getting head of incidents dataset
   incident_start_time incident_edge            incident_type  \
0               7613.0   436943754#0  multi_vehicle_collision   
1              15942.0   436791121#0          stalled_vehicle   
2              23413.0   512810351#0  multi_vehicle_collision   
3              31451.0    -436789319  multi_vehicle_collision   
4              41274.0   533573789#0          stalled_vehicle   

   accident_duration  incident_lane  
0             5680.0              2  
1              983.0  436791121#0_0  
2             3496.0              2  
3             3227.0   -436789319_2  
4             1938.0  533573789#0_1  
Number of incidents in this simulation 20
Number of incidents in this simulation with less severe incidents 9
Number of incidents in this simulation with more severe incidents 11
Input traffic raw data shape is (82944000, 18)


In [23]:
df_traffic_reduced = df_traffic_reduced.drop(["vehicles_per_lane_1","vehicles_per_lane_0","lane_mean_speed_0","lane_mean_speed_1"],axis=1)

In [24]:

if NUM_OF_JUNCTIONS == 3:
    print(f" Number of junctions considered 3")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","436943762#0"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2"]
    
elif NUM_OF_JUNCTIONS == 4:
    print(f" Number of junctions considered 4")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767"]
    
elif NUM_OF_JUNCTIONS == 5:
    print(f" Number of junctions considered 5")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0","436794669","436942357"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767","-436794676#1","531969915#0"]
    
elif NUM_OF_JUNCTIONS == 6:
    print(f" Number of junctions considered 6")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0","436794669","436942357","-436942356#1","436942362#0","436789580#1"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767","-436794676#1","531969915#0","436942356#0","-436942362#3","436789564#0"]
    
elif NUM_OF_JUNCTIONS == 7:
    print(f" Number of junctions considered 7")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0","436794669","436942357","-436942356#1","436942362#0","436789580#1","436794670","1051038541#0","-436794679#3"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767","-436794676#1","531969915#0","436942356#0","-436942362#3","436789564#0","A-1088637809#1","A436942385#0","A436794679#0"]
    
elif NUM_OF_JUNCTIONS == 8:
    print(f" Number of junctions considered 8")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0","436794669","436942357","-436942356#1","436942362#0","436789580#1","436794670","1051038541#0","-436794679#3","436943745#0","351673438","-613687451#1"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767","-436794676#1","531969915#0","436942356#0","-436942362#3","436789564#0","-1088637809#1","436942385#0","436794679#0","-436943745#2","436943742","436943774"]
    

else:
    print(f" Number of junctions considered {NUM_OF_JUNCTIONS}")
    print("Entered too many sensors resorting to 8 sensors")
    incoming_edges = ["436794680#0","436791113#0","-436942381#3","533371302#0","436942374","436790491","436943750#0","-643913497","436940278","436943762#0","436794669","436942357","-436942356#1","436942362#0","436789580#1","436794670","1051038541#0","-436794679#3","436943745#0","351673438","-613687451#1"]
    outgoing_edges = ["533573776#0","5607328#0","436942381#0","436790495","-1033824750","30031286#0","436943743#0","-436943762#2","436940270","519448767","-436794676#1","531969915#0","436942356#0","-436942362#3","436789564#0","-1088637809#1","436942385#0","436794679#0","-436943745#2","436943742","436943774"]
    

 Number of junctions considered 8


In [25]:
edges = incoming_edges + outgoing_edges

In [26]:
count = 0
df = df_traffic_reduced[df_traffic["identified_edge"] == edges[0] ]
df = df.reset_index(drop=True)
df[f"rolling_junction_mean_speed_{count}"] = df['junction_mean_speed'].rolling(ROLLING_WINDOW_LENGTH).mean()
df[f"rolling_traffic_occupancy_{count}"] = df['traffic_occupancy'].rolling(ROLLING_WINDOW_LENGTH).mean()
df[f"rolling_traffic_count_{count}"] = df['traffic_count'].rolling(ROLLING_WINDOW_LENGTH).mean()

Y = df[["step","time_of_day","incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label"]]
df_data = df[["step",f"rolling_junction_mean_speed_{count}",f"rolling_traffic_count_{count}",f"rolling_traffic_occupancy_{count}"]]
count+=1

/tmp/ipykernel_396593/770496982.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df_traffic_reduced[df_traffic["identified_edge"] == edges[0] ]


In [27]:
a = df.shape

print("Shape so far",a[0],a[1])

Shape so far 1296000 17


In [28]:
for index,edge in enumerate(edges):
    
    if index == 0:
        continue
    print(f"Processed {count}")
    df = df_traffic_reduced[df_traffic_reduced["identified_edge"] == edge ]
    df = df.reset_index(drop=True)
    df[f"rolling_junction_mean_speed_{count}"] = df['junction_mean_speed'].rolling(ROLLING_WINDOW_LENGTH).mean()
    df[f"rolling_traffic_count_{count}"] = df['traffic_count'].rolling(ROLLING_WINDOW_LENGTH).mean()
    df[f"rolling_traffic_occupancy_{count}"] = df['traffic_occupancy'].rolling(ROLLING_WINDOW_LENGTH).mean()
    
    df_temp = df[["step",f"rolling_junction_mean_speed_{count}",f"rolling_traffic_count_{count}",f"rolling_traffic_occupancy_{count}"]]
    df_data = pd.merge(left = df_data, right = df_temp , on = "step", how = "inner")

    count+=1
    

Processed 1
Processed 2
Processed 3
Processed 4
Processed 5
Processed 6
Processed 7
Processed 8
Processed 9
Processed 10
Processed 11
Processed 12
Processed 13
Processed 14
Processed 15
Processed 16
Processed 17
Processed 18
Processed 19
Processed 20
Processed 21
Processed 22
Processed 23
Processed 24
Processed 25
Processed 26
Processed 27
Processed 28
Processed 29
Processed 30
Processed 31
Processed 32
Processed 33
Processed 34
Processed 35
Processed 36
Processed 37
Processed 38
Processed 39
Processed 40
Processed 41


In [29]:
df_traffic_data = pd.merge(left = df_data, right = Y , on = "step", how = "inner")

In [35]:
for i in df_traffic_data.columns:
    print(i)

step
rolling_junction_mean_speed_0
rolling_traffic_count_0
rolling_traffic_occupancy_0
rolling_junction_mean_speed_1
rolling_traffic_count_1
rolling_traffic_occupancy_1
rolling_junction_mean_speed_2
rolling_traffic_count_2
rolling_traffic_occupancy_2
rolling_junction_mean_speed_3
rolling_traffic_count_3
rolling_traffic_occupancy_3
rolling_junction_mean_speed_4
rolling_traffic_count_4
rolling_traffic_occupancy_4
rolling_junction_mean_speed_5
rolling_traffic_count_5
rolling_traffic_occupancy_5
rolling_junction_mean_speed_6
rolling_traffic_count_6
rolling_traffic_occupancy_6
rolling_junction_mean_speed_7
rolling_traffic_count_7
rolling_traffic_occupancy_7
rolling_junction_mean_speed_8
rolling_traffic_count_8
rolling_traffic_occupancy_8
rolling_junction_mean_speed_9
rolling_traffic_count_9
rolling_traffic_occupancy_9
rolling_junction_mean_speed_10
rolling_traffic_count_10
rolling_traffic_occupancy_10
rolling_junction_mean_speed_11
rolling_traffic_count_11
rolling_traffic_occupancy_11
rolli

In [30]:
a = df_traffic_data.shape

print("Shape so far",a[0],a[1])

Shape so far 1296000 135


In [31]:
# Dealing with Vehicle data
dtype = {
'step':'int64',
'time_of_day':'int64',
'car_id':'object',
'identified_edge':'object',
'identified_lane':'float64',
'junction_mean_speed':'float64',
'vehicle_speed':'float64',
'vehicle_acceleration':'float64'
}
df_vehicle = pd.read_csv(eval_vehicle,dtype=dtype)

In [32]:
df_vehicle.head()

,step,time_of_day,car_id,identified_edge,identified_lane,vehicle_speed,vehicle_acceleration,junction
0,7,7,veh7,-436942356#1,1.0,0.000000,0.000000,6
1,7,7,veh7.1,-436942356#1,2.0,0.000000,0.000000,6
2,8,8,veh7,-436942356#1,1.0,1.433850,1.433850,6
3,8,8,veh7.1,-436942356#1,2.0,2.020224,2.020224,6
4,9,9,veh7,-436942356#1,1.0,4.033344,2.599494,6


In [36]:
df_vehicle = df_vehicle[~df_vehicle['identified_edge'].isin(excess_list)]

In [39]:
df_vehicle.head()

,step,time_of_day,car_id,identified_edge,identified_lane,vehicle_speed,vehicle_acceleration
0,7,7,veh7,-436942356#1,1.0,0.000000,0.000000
1,7,7,veh7.1,-436942356#1,2.0,0.000000,0.000000
2,8,8,veh7,-436942356#1,1.0,1.433850,1.433850
3,8,8,veh7.1,-436942356#1,2.0,2.020224,2.020224
4,9,9,veh7,-436942356#1,1.0,4.033344,2.599494


In [38]:
df_vehicle = df_vehicle.drop(["junction"],axis=1)

In [40]:
# sensor details

sensors = {

    1: "5607328#0",
    2:"436791113#0",
    3:"533573776#0",
    4:"436794680#0",
    5:"436794670",
    6:"436942385#0",
    7:"1051038541#0",
    8:"436794679#0",
    9:"-436794679#3",
    10:"-1088637809#1",
    11:"436794669",
    12:"531969915#0",
    13:"436942357",
    14:"-436794676#1",
    15:"436790495",
    16:"533371302#0",
    17:"-1033824750",
    18:"436942374",
    19:"30031286#0",
    20:"436790491",
    21:"436942381#0",
    22:"-436942381#3",
    23:"436789580#1",
    24:"436942356#0",
    25:"-436942356#1",
    26:"-436942362#3",
    27:"436942362#0",
    28:"436789564#0",
    29:"436940278",
    30:"436940270",
    31:"-643913497",
    32:"519448767",
    33:"436943745#0",
    34:"436943742",
    35:"351673438",
    36:"436943774",
    37:"-613687451#1",
    38:"-436943745#2",
    39:"436943750#0",
    40:"-436943762#2",
    41:"436943762#0",
    42:"436943743#0"

}

In [41]:
junctions_sensor_combo ={

    3:[[3,22],[21,4],[1,16],[15,2],[40,20],[19,41],[42,18],[17,39],[3,41],[42,2],[1,39],[40,4]],

    4:[[3,22],[21,4],[1,16],[15,2],[40,20],[19,41],[42,18],[17,39],[3,41],[42,31],[30,39],[40,4],[32,2],[1,29],[32,16],[15,29],[30,18],[17,31]],
    
    5:[[3,11],[14,4],[30,39],[42,31],[32,2],[1,29],[40,13],[12,41],[3,22],[14,22],[21,4],[21,11],[1,16],[15,2],[19,13],[12,20],[32,16],[15,29],[30,18],[17,31],[17,39],[42,18],[19,41],[40,20]],

    6:[[3,11],[14,4],[30,39],[42,31],[32,2],[1,29],[26,13],[12,27],[40,25],[24,41],[3,22],[14,22],[21,4],[21,11],[1,16],[15,2],[19,23],[28,20],[32,16],[15,29],[30,18],[17,31],[17,39],[42,18]],

    7:[[3,5],[10,4],[8,11],[14,9],[30,39],[42,31],[32,2],[1,29],[26,13],[12,27],[40,25],[24,41],[6,22],[21,7],[1,16],[15,2],[19,23],[28,20],[32,16],[15,29],[30,18],[17,31],[17,39],[42,18]],

    8:[[3,5],[10,4],[8,11],[14,9],[30,33],[38,31],[34,39],[42,35],[32,2],[1,29],[26,13],[12,27],[40,25],[24,41],[6,22],[21,7],[1,16],[15,2],[19,23],[28,20],[32,16],[15,29],[36,18],[17,37]]

}

In [42]:
interested_combo = junctions_sensor_combo[NUM_OF_JUNCTIONS]

In [43]:
print(interested_combo)
print(len(interested_combo))

[[3, 5], [10, 4], [8, 11], [14, 9], [30, 33], [38, 31], [34, 39], [42, 35], [32, 2], [1, 29], [26, 13], [12, 27], [40, 25], [24, 41], [6, 22], [21, 7], [1, 16], [15, 2], [19, 23], [28, 20], [32, 16], [15, 29], [36, 18], [17, 37]]
24


In [44]:
steps = [i for i in range(LENGTH_OF_SIM)]
data_dd = {"steps":steps}
dd_dataframe = pd.DataFrame(data_dd)
dd_dataframe.shape

(1296000, 1)

In [45]:
count = 1
for sensor_combination in interested_combo:
    print(count)
    outgoing_sensor = sensors[sensor_combination[0]]
    incoming_sensor = sensors[sensor_combination[1]]
    df_outgoing = df_vehicle[df_vehicle['identified_edge'] == outgoing_sensor]
    df_incoming = df_vehicle[df_vehicle['identified_edge'] == incoming_sensor]
    df_incoming = df_incoming.reset_index(drop=True)
    df_outgoing = df_outgoing.reset_index(drop=True)

    df_outgoing_unique = df_outgoing[df_outgoing["car_id"].notnull()]
    df_incoming_unique = df_incoming[df_incoming["car_id"].notnull()]

    df_outgoing_unique = df_outgoing_unique.groupby('car_id').first().reset_index()
    df_incoming_unique = df_incoming_unique.groupby('car_id').first().reset_index()

    df_outgoing_unique = df_outgoing_unique.sort_values(by = 'step')
    df_incoming_unique = df_incoming_unique.sort_values(by = 'step')
    
    df_merged = pd.merge(left = df_incoming_unique,right = df_outgoing_unique, on='car_id', how='inner')

    df_merged["travel_time"] = df_merged["step_x"] - df_merged["step_y"]

    df_travel_time_data = df_merged[["step_x","travel_time"]]

    
    df_travel_time_data2 = pd.merge(left = df_travel_time_data,right = dd_dataframe, right_on='steps',left_on="step_x", how='right')

    df_travel_time_data2[f"rolling_travel_time_{sensor_combination[0]}_{sensor_combination[1]}"] = df_travel_time_data2["travel_time"].rolling(300,min_periods=1).mean()

    df_travel_time_data2 = df_travel_time_data2.drop(["step_x","travel_time"],axis=1)
    df_traffic_data = pd.merge(left = df_traffic_data, right = df_travel_time_data2 , left_on = "step",right_on = "steps", how = "inner")
    df_traffic_data = df_traffic_data.drop(["steps"],axis=1)
    count+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [46]:
df_traffic_data.shape

(1699815, 159)

In [47]:
df_traffic_data.head()

,step,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,...,rolling_travel_time_6_22,rolling_travel_time_21_7,rolling_travel_time_1_16,rolling_travel_time_15_2,rolling_travel_time_19_23,rolling_travel_time_28_20,rolling_travel_time_32_16,rolling_travel_time_15_29,rolling_travel_time_36_18,rolling_travel_time_17_37
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df = df_traffic_data[315:]
Y = df["accident_label"]
X = df.drop(["incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label"],axis=1)

In [57]:
def standardize(col):
   return (col - col.mean()) / col.std()

X = X.apply(standardize)

In [73]:
file_name = "/home/local/ASURITE/speddira/dev/git_repo/traffic_sense_net/city_scale/xgboost/saved_models/V1/xg_model.pkl"
xgb_model_loaded = pickle.load(open(file_name, "rb"))

In [59]:
X = X.drop(["step"],axis=1)

In [60]:
X.head()

,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,rolling_junction_mean_speed_3,...,rolling_travel_time_6_22,rolling_travel_time_21_7,rolling_travel_time_1_16,rolling_travel_time_15_2,rolling_travel_time_19_23,rolling_travel_time_28_20,rolling_travel_time_32_16,rolling_travel_time_15_29,rolling_travel_time_36_18,rolling_travel_time_17_37
315,-3.728238,-1.187982,-1.192763,-5.071410,-1.780472,-1.832412,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,NaN,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
316,-3.706246,-1.184480,-1.189292,-5.030042,-1.775693,-1.825807,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,NaN,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
317,-3.684328,-1.180979,-1.185820,-4.993713,-1.768526,-1.818162,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.479907,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
318,-3.684328,-1.180979,-1.185820,-4.993713,-1.768526,-1.818162,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.479907,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
319,-3.662442,-1.177478,-1.182349,-4.960124,-1.758970,-1.808151,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.479907,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298


In [70]:
for col in X.columns:
    
    print(col)

rolling_junction_mean_speed_0
rolling_traffic_count_0
rolling_traffic_occupancy_0
rolling_junction_mean_speed_1
rolling_traffic_count_1
rolling_traffic_occupancy_1
rolling_junction_mean_speed_2
rolling_traffic_count_2
rolling_traffic_occupancy_2
rolling_junction_mean_speed_3
rolling_traffic_count_3
rolling_traffic_occupancy_3
rolling_junction_mean_speed_4
rolling_traffic_count_4
rolling_traffic_occupancy_4
rolling_junction_mean_speed_5
rolling_traffic_count_5
rolling_traffic_occupancy_5
rolling_junction_mean_speed_6
rolling_traffic_count_6
rolling_traffic_occupancy_6
rolling_junction_mean_speed_7
rolling_traffic_count_7
rolling_traffic_occupancy_7
rolling_junction_mean_speed_8
rolling_traffic_count_8
rolling_traffic_occupancy_8
rolling_junction_mean_speed_9
rolling_traffic_count_9
rolling_traffic_occupancy_9
rolling_junction_mean_speed_10
rolling_traffic_count_10
rolling_traffic_occupancy_10
rolling_junction_mean_speed_11
rolling_traffic_count_11
rolling_traffic_occupancy_11
rolling_ju

In [61]:
X.head(20)

,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,rolling_junction_mean_speed_3,...,rolling_travel_time_6_22,rolling_travel_time_21_7,rolling_travel_time_1_16,rolling_travel_time_15_2,rolling_travel_time_19_23,rolling_travel_time_28_20,rolling_travel_time_32_16,rolling_travel_time_15_29,rolling_travel_time_36_18,rolling_travel_time_17_37
315,-3.728238,-1.187982,-1.192763,-5.071410,-1.780472,-1.832412,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,NaN,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
316,-3.706246,-1.184480,-1.189292,-5.030042,-1.775693,-1.825807,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,NaN,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
317,-3.684328,-1.180979,-1.185820,-4.993713,-1.768526,-1.818162,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.479907,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
318,-3.684328,-1.180979,-1.185820,-4.993713,-1.768526,-1.818162,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.479907,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
319,-3.662442,-1.177478,-1.182349,-4.960124,-1.758970,-1.808151,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.479907,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
320,-3.640556,-1.173977,-1.178878,-4.928480,-1.749414,-1.798958,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.454277,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
321,-3.618671,-1.170476,-1.175406,-4.901419,-1.739858,-1.788765,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.454277,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
322,-3.596786,-1.166975,-1.171935,-4.876360,-1.730302,-1.778572,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.454277,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
323,-3.574902,-1.163474,-1.168464,-4.854382,-1.720746,-1.768379,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.454277,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298
324,-3.553017,-1.159973,-1.164992,-4.830681,-1.711190,-1.758187,-1.949403,-0.890003,-0.917971,-2.493723,...,-0.323106,-0.718592,NaN,-0.454277,-0.436527,-0.281413,NaN,NaN,-0.512002,-0.761298


In [92]:
import numpy as np
def callme(prediction_rate):
    history = [0]  # Assuming no incident at start
    incident_counter = 0
    detected_counter = 0
    new_incident_flag = False
    incident_detected_flag = False
    predictions_count = 0
    i = 0  # Time since last new incident
    false_alarm_counter = 0
    detection_times = []  # To calculate mean time for detection
    counter = 1
    for (index,row),(yrow) in zip(X.iterrows(),Y.to_numpy()):
        
        if counter% prediction_rate == 0:
            
            predictions_count+=1
            predicted = xgb_model_loaded.predict([row.values])[0] 
            # print(predicted)
            
            if len(history) > 0:
                previous_label = history[-1]
            else:
                previous_label = 0  # Default to no incident for the first element

            # Update history
            current_label = 1 if yrow else 0
            history.append(current_label)

            # New incident started
            if previous_label == 0 and current_label == 1:
                incident_counter += 1
                new_incident_flag = True
                incident_detected_flag = False  # Reset detection flag for new incident
                i = 0  # Reset timer for new incident
                print("New incident started")

            # Incident detected
            if new_incident_flag and predicted == 0 and not incident_detected_flag:
                detected_counter += 1
                detection_times.append(i)
                incident_detected_flag = True
                print(f"Incident Detected by model after {i} timesteps.")

            # Incident finished
            if previous_label == 1 and current_label == 0:
                new_incident_flag = False
                print("Incident Finished")


            # False alarm raised
            if not new_incident_flag and predicted == 0 and not (1 in history[-1000:]):
                false_alarm_counter += 1
                
        # Increment timer if incident ongoing but not yet detected
        if new_incident_flag and not incident_detected_flag:
            i += 1
        counter+=1
        
        
    arr = np.array(detection_times)
    row = {"prediction_rate" : prediction_rate, 
           "incident_counter": incident_counter,
           "detected_counter":detected_counter,
           "mttd":arr.mean(),
           "false_alarm_counter":false_alarm_counter,
           "predictions_count":predictions_count
        
    }    
    return row
                # print("False Alarm raised by model.")

In [93]:
# incident_counter = 0
# detected_counter = 0
# new_incident_flag = False
# incident_detected_flag = False
# predictions_count = 0
# i = 0  # Time since last new incident
# false_alarm_counter = 0
# detection_times = []  # To calculate mean time for detection
import numpy as np
rows = []
experiments = [5,10,30,60,90,120]

for i in experiments:
    print("Running experiment on ",i)
    rows.append(callme(i))

Running experiment on  5
New incident started
Incident Detected by model after 110 timesteps.
Incident Finished
New incident started
Incident Detected by model after 5 timesteps.
Incident Finished
New incident started
Incident Detected by model after 1195 timesteps.
Incident Finished
New incident started
Incident Detected by model after 250 timesteps.
Incident Finished
New incident started
Incident Detected by model after 0 timesteps.
Incident Finished
New incident started
Incident Detected by model after 0 timesteps.
Incident Finished
New incident started
Incident Detected by model after 45 timesteps.
Incident Finished
New incident started
Incident Finished
New incident started
Incident Detected by model after 245 timesteps.
Incident Finished
New incident started
Incident Finished
New incident started
Incident Detected by model after 195 timesteps.
Incident Finished
New incident started
Incident Detected by model after 100 timesteps.
Incident Finished
New incident started
Incident Det

In [ ]:
results = pd.DataFrame(rows)

In [ ]:
results["false_alarm_rate"] = 100*results["false_alarm_counter"]/results["predictions_count"]
results["accuracy"] = 100*results["detected_counter"]/results["incident_counter	"]

In [ ]:
results.head()

,incident_counter,detected_counter,mttd,false_alarm_counter,predictions_count,false_alarm_rate
0,20,18,58.388889,92204,339900,27.126802
1,20,18,33.444444,44328,169950,26.082966
2,20,16,12.750000,14670,56650,25.895852
3,20,16,6.500000,7204,28325,25.433363
4,20,16,4.375000,4623,18883,24.482339
